In [42]:
import numpy as np
import xarray as xr
import scores
from scipy.stats import skewnorm

from scores.probability import (
    twcrps_for_ensemble,
    tail_twcrps_for_ensemble,
    crps_cdf,
    crps_for_ensemble,
)

In [3]:
# fcst= xr.DataArray(
#     data=[[0.0, 4, 3, 7], [1, -1, 2, 4], [0, 1, 4, np.nan], [2, 3, 4, 1], [2, np.nan, np.nan, np.nan]],
#     dims=["stn", "ens_member"],
#     coords={"stn": [101, 102, 103, 104, 105], "ens_member": [1, 2, 3, 4]},
# )
# obs = xr.DataArray(data=[2.0, 3, 1, np.nan, 4], dims=["stn"], coords={"stn": [101, 102, 103, 104, 105]})

# weights = xr.DataArray(data=[0, 1, 3, 3, 3], dims=["stn"], coords={"stn": [101, 102, 103, 104, 105]})

-2.4725074438208607

In [60]:
a = 4  # Skewness parameter
size = 10000  # Number of values
# Generate skew normal distribution
data = skewnorm.rvs(a, size=size)
data = data * 5

fcst = xr.DataArray(
    data=data,
    dims=["ens_member"],
    coords={"ens_member": np.arange(0, size)},
)

obs = xr.DataArray(data=[5], dims=["time"], coords={"time": [0]})

weights = xr.DataArray(
    data=[
        [0, 0, 1, 1],  # fcst has nan
    ],
    dims=["time", "threshold"],
    coords={"time": [0], "threshold": [0, 4.999999999, 5, 30]},
)

# Step 1: Sort the ensemble members
sorted_fcst = fcst.sortby(fcst)

# Step 2: Create cumulative probabilities
n = sorted_fcst.size
cumulative_probs = xr.DataArray(
    np.arange(1, n + 1) / n,
    dims=["ens_member"],
    coords={"ens_member": sorted_fcst["ens_member"]},
)

# Step 3: Create the CDF as a DataArray
cdf = xr.DataArray(
    data=cumulative_probs,
    dims=["threshold"],
    coords={"threshold": sorted_fcst.data},
)

print(cdf)

# cdf = cdf.expand_dims("time")
cdf = cdf.assign_coords(time=0)
cdf = cdf.expand_dims("time")

<xarray.DataArray (threshold: 10000)>
array([1.000e-04, 2.000e-04, 3.000e-04, ..., 9.998e-01, 9.999e-01,
       1.000e+00])
Coordinates:
  * threshold  (threshold) float64 -3.369 -3.265 -3.247 ... 18.26 18.46 19.39


In [54]:
crps_cdf(cdf, obs).total

<xarray.DataArray 'total' ()>
array(1.04855105)

In [55]:
crps_for_ensemble(fcst, obs, ensemble_member_dim="ens_member")

<xarray.DataArray ()>
array(1.04843937)

In [56]:
crps_for_ensemble(fcst, obs, ensemble_member_dim="ens_member", method="fair")

<xarray.DataArray ()>
array(1.04826323)

In [57]:
tail_twcrps_for_ensemble(fcst, obs, ensemble_member_dim="ens_member", threshold=5)

<xarray.DataArray ()>
array(0.14675999)

In [62]:
tail_twcrps_for_ensemble(
    fcst, obs, ensemble_member_dim="ens_member", threshold=5, method="fair"
)

<xarray.DataArray ()>
array(0.14161894)

In [61]:
crps_cdf(cdf, obs, threshold_weight=weights).total

<xarray.DataArray 'total' ()>
array(0.14160481)

In [4]:
def vary_weights(x):
    return np.maximum(x, weights)


twcrps_for_ensemble(fcst, obs, "ens_member", vary_weights, preserve_dims="all")

<xarray.DataArray (stn: 5)>
array([1.125     , 0.875     , 0.11111111,        nan, 1.        ])
Coordinates:
  * stn      (stn) int64 101 102 103 104 105

In [5]:
tail_twcrps_for_ensemble(
    fcst, obs, "ens_member", tail="upper", threshold=0, preserve_dims="all"
)

<xarray.DataArray (stn: 5)>
array([1.125     , 0.9375    , 0.44444444,        nan, 2.        ])
Coordinates:
  * stn      (stn) int64 101 102 103 104 105

In [8]:
tail_twcrps_for_ensemble(
    fcst, obs, "ens_member", tail="lower", threshold=weights, preserve_dims="all"
)

ValueError: 'lofwer' is not one of 'upper' or 'lower'